In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [2]:
import time
from tqdm import tqdm
import pathlib
import google.generativeai as genai
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
from PIL import Image
import pytesseract
from IPython.display import Markdown

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Apollo 11 Mission

Different types of data (otherwise known as multimodal) from NASA's public archive.

1. Text: Full final NASA report post-mission, spanning over 300
pages of incredibly informative content that details a summary of everything
that happened as well as conclusions that NASA researchers and engineers
came to.

2. Video: Several clips of the famous Neil Armstrong and Buzz Aldrin footage as they
first stepped onto the moon, containing highlights of their moonwalks as well
as raising the American flag.

3. Audio: Audio recorded throughout the
mission, which provides insights into how communication between the astronauts
occurred as well as from the astronauts to mission control.

## Data Preparation

Upload the resources.zip folder and unzip it using the following commands:

In [4]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-08-05 06:08:03--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.138.72, 104.19.141.72, 104.19.139.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.138.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M   178MB/s    in 1.5s    

2024-08-05 06:08:05 (178 MB/s) - ‘resources.zip’ saved [286142532/286142532]



In [5]:
!unzip resources.zip

Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl


When working with different types of data,
we first need to parse it in a way that Gemini can understand. We will prepare our data by extracting all file names from the `resources` directory.

In [6]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [7]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/text/images-333.jpg
resources/text/images-020.jpg
resources/text/images-023.jpg
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/video/Apollo11MoonwalkMontage.mov
resources/video/BuzzDescendsCompilation.mov
resources/video/Apollo11PlaqueComparison.mov
resources/video/RaisingTheAmericanFlag.mov
resources/video/Apollo11Intro.mov
resources/video/OneSmallStepCompilation.mov
14


## Retrieval Augmented Generation (RAG)

To showcase how we build a RAG, we will first build one for the Text case, and generalize it further after. Here is the general idea:
1. **Data Preparation** (done above): We first collected various types of data from NASA's public archive related to the Apollo 11 mission, including text, video, and audio files.
2. **Data Extraction and Summarization**: Extract the multimodal data from images, e.g. extract text from images using Optical Character Recognition (OCR), and use Gemini to generate summaries using a specialized prompt.
3. **Embedding Generation**: Convert the generated summaries into vector embeddings using Gemini's Text Embedding Model. These embeddings represent the summaries in a numerical format suitable for efficient similarity searches.
4. **Creating a Vector Database**: A Vector database was created to store the embeddings. This database facilitates fast and efficient retrieval of relevant documents based on similarity searches. We chose to use Chroma DB.
5. **Querying the RAG System**: For a given query, the system retrieves the most relevant documents (based on their embeddings) and generates a response using the retrieved documents as context.

Something important to note is that RAGs are usually used only when there is a surplus of data. In other words, if the data can't fit into the model prompt. In this case, the data we provided likely can fit into Gemini's 1 million token window, but for the sake of simplicity and restrictions of Google Colab's runtime, we opted to use a smaller set of data.

### Text

We will use Tesseract OCR (Optical Character Recognition) to extract text from images of the NASA report.

In [8]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

Let's create a function to take in our images of a PDF, transcribe them into text, and summarize each of them.

In [9]:
def create_text_summary():
  path = pathlib.Path("resources/text")

  text_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

  images = []
  text_summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue

    image = Image.open(f)
    response = model.generate_content([text_summary_prompt, pytesseract.image_to_string(image)])

    images.append(image)
    text_summaries.append(response.text)

  return images, text_summaries

In [10]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)

In [11]:
image_files, text_summaries = create_text_summary()

Now, we can check out the generated summaries of the three pages we have!

In [12]:
for text_summary in text_summaries:
  print(text_summary)

This document outlines ground rules and assumptions for EPS analysis. It covers the activation timing of descent stage batteries, checkout time in lunar orbit, battery paralleling during powered descent, operational status of S-band equipment, rendezvous radar electronics, primary navigation and guidance subsystem, and forward window heaters. 

This document outlines the Apollo 11 mission plan, including crew activities and operations for the AS-506/CSM-107/LM-5 spacecraft.  The plan is based on a July 16, 1969 launch and uses trajectory parameters defined in the Apollo Mission G Spacecraft Operational Trajectory.  The document is under configuration control and all proposed changes must be submitted to the Crew Procedures Control Board (CPCB) via a Crew Procedures Change Request.  Contact Mr. T. A. Guillory for coordination of proposed changes and Mr. W. J. North for distribution requests. 

This document describes the mission launch and timeline for the Apollo 11 lunar landing missio

We create the Chroma database using the generated summaries.

**Vector Database**: A specialized database designed to store and manage high-dimensional vectors, which are numerical representations of data points. It allows efficient similarity searches to find vectors (and their corresponding data) that are close to a given query vector.

**Chroma DB**: An implementation of a vector database used to store and retrieve vector embeddings. These embeddings are generated from our summaries and allow us to perform efficient similarity searches.


In [13]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [14]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [15]:
text_db = create_chroma_db(text_summaries, "text_nasa")

Let's also take a peak at the `text_db` and ensure that embeddings were generated:

In [16]:
data = {
    'embeddings': text_db.peek()['embeddings'],
    'documents': text_db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.0712694451212883, -5.655638233292848e-06, -...",This document outlines ground rules and assump...
1,"[0.07237721234560013, 0.024491339921951294, 0....",This document outlines the Apollo 11 mission p...
2,"[0.07501331716775894, -0.020817341282963753, 0...",This document describes the mission launch and...


You should see a column called `embeddings` with what are seemingly random values, but these values are actually high-dimensional vectors that represent the semantic meaning of your summaries.

Now let's actually try querying our information. We'll test a simple example like getting some file that has to do with the Apollo 11 Flight Plan.

In [17]:
def get_relevant_files(query, db):
  results = db.query(query_texts=[query], n_results=3)
  return results["ids"][0]

In [18]:
files = get_relevant_files("Apollo 11 Flight Plan", text_db)
print(files)

['1', '2', '0']


You should expect to see something like `['1', '0', '2']`. This means that the first entry in the `text_db` is most similar. If you look above at our `pd.DataFrame` output, the document with id 1 is the document about the Apollo 11 Flight Plan, so this is working as we expected!

### Video and Audio


In [19]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)])

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file])

    files.append(file)
    summaries.append(response.text)

  return files, summaries

Now, we will create a folder with all of our data of different modalities. In particular, the first 5 are audio files, next 3 are text files, and final 6 are video files.

In [20]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/text/images-333.jpg
resources/text/images-020.jpg
resources/text/images-023.jpg
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...
resources/video/Apollo11Intro.mov
Waiting for video file upload...
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...


In [21]:
db = create_chroma_db(all_summaries, "nasa")

Again, ensure that the embeddings were generated. Notice that now, we have audio, video, and text data.

In [22]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.056587886065244675, -0.02480756677687168, -...",Crew members discuss the pressure and other da...
1,"[0.03179985284805298, -0.010213697329163551, -...",Two people are discussing a flight plan. One p...
2,"[0.052620116621255875, -0.015188580378890038, ...",Apollo 11 astronauts Buzz Aldrin and Neil Arms...
3,"[0.04529229924082756, -0.01137058436870575, -0...",This video shows a side-by-side comparison of ...
4,"[0.04843682795763016, -0.03361263498663902, 0....",This is footage from the Apollo 11 moon landin...
5,"[0.03611447289586067, -0.03427395224571228, -0...",This video is a comparison of the original NAS...
6,"[0.0539969727396965, 0.026326457038521767, -0....",This audio is a conversation between two peopl...
7,"[-0.00819326564669609, 0.0029475842602550983, ...",Two people are talking about flying and how to...
8,"[0.04129211977124214, -0.02891802228987217, -0...",A group of astronauts are on a mission in spac...
9,"[0.08152654767036438, 0.000555358303245157, -0...",This document outlines ground rules and assump...


In [23]:
files = get_relevant_files("communication with Mission Control", db)
print(files)

['0', '6', '8']


Can we do more than just return the most relevant file? Yes we can! We can ask Gemini to return a response to the query using the files it thinks are most relevant, provide an answer and tell us what files it used.

In [24]:
def query_rag(query, db):
    files = get_relevant_files(query, db)
    prompt = [all_files[int(f)] for f in files]
    prompt.append("Generate a response to the query using the provided files. Here is the query.")
    prompt.append(query)
    return model.generate_content(prompt).text, [all_file_names[int(f)] for f in files]

In [25]:
for response in query_rag("Explain what happened with the Apollo 11 Mission.", db):
    print(response)

The Apollo 11 Mission launched on July 16, 1969. The mission was a monumental event in human history, as it marked the first time humans walked on the moon. It was part of the US government's efforts to beat the Soviet Union in the space race. 

The mission involved three astronauts, Neil Armstrong, Buzz Aldrin, and Michael Collins, aboard the Apollo 11 spacecraft. On July 20, 1969, Armstrong and Aldrin became the first humans to walk on the moon's surface. Armstrong famously said, "That's one small step for man, one giant leap for mankind."

The astronauts spent several hours on the moon, taking photographs, collecting lunar samples, planting the American flag, and leaving a plaque commemorating the mission. They also conducted scientific experiments before returning to the lunar module and ascending to the command module. Collins remained in lunar orbit during this time.

The mission concluded on July 24, 1969, when the astronauts splashed down safely in the Pacific Ocean. The Apollo

In [26]:
for response in query_rag("What happens at the Translunar Coast in the Mission Description?", db):
    print(response)

The Mission Description does not mention a Translunar Coast. 
['resources/video/Apollo11MoonwalkMontage.mov', 'resources/audio/Apollo11OnboardAudioHighlightClip4.mp3', 'resources/audio/Apollo11OnboardAudioHighlightClip1.mp3']


A full end to end multimodal RAG.